In [1]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt

import os
from PIL import Image
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
from sklearn.model_selection import train_test_split
import torchvision.models as models
import torch.optim as optim
import torchvision.transforms as transforms
from tqdm import tqdm


%matplotlib inline

/Users/samvasserman/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
file_path = './Data/list_attr_celeba.txt'
file_content = ''

with open(file_path, "r") as file:
    # Read the entire file content into a single string
    file_content = file.read()
    file_content = file_content.replace('  ', ',')
    file_content = file_content.replace(' -', ',-')
    file_content = file_content.replace(' ', ',')


file_path = "attributes.csv"

with open(file_path, "w") as file:
    file.write(file_content)

In [11]:
df = pd.read_csv('./attributes.csv')

In [12]:
df = df.replace(-1, 0)

In [76]:
train_df, test_df = train_test_split(df, train_size=10000, random_state=42)

In [77]:
train_df

,Image,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
65294,065295.jpg,0,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
136394,136395.jpg,0,0,0,1,1,0,1,1,0,...,0,1,0,0,0,0,0,0,1,0
125758,125759.jpg,1,0,1,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,1
181303,181304.jpg,0,1,1,0,0,0,1,0,1,...,0,0,0,1,0,0,1,0,0,1
142658,142659.jpg,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,119880.jpg,0,0,1,0,0,0,0,0,0,...,0,1,1,0,0,0,1,0,0,1
103694,103695.jpg,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
131932,131933.jpg,0,0,1,1,0,0,0,0,0,...,0,1,1,0,0,0,1,1,0,1
146867,146868.jpg,0,0,1,1,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,1


In [78]:
class CustomImageDataset(Dataset):
    def __init__(self, data_dir, dataframe):
        self.data_dir = data_dir
        self.dataframe = dataframe
        self.image_paths = self.dataframe['Image'].tolist()
        self.labels = self.dataframe.drop('Image', axis=1).values.astype(np.float32)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_name = os.path.join(self.data_dir, self.image_paths[idx])
        image = Image.open(img_name)
        transform = transforms.Compose([transforms.ToTensor()])
        
        image = transform(image)

        label = torch.tensor(self.labels[idx])
        return image, label

In [79]:
train_dataset = CustomImageDataset('./Data/img_align_celeba/', train_df)

In [80]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [89]:
resnet18 = models.resnet18(pretrained=True) # load in the pre trained resnet18 model

In [90]:
num_labels_train = len(train_df.columns) - 1
num_labels_test = len(test_df.columns) - 1
print(num_labels_train, num_labels_test)

40 40


In [91]:
in_features = resnet18.fc.in_features

num_attributes = 40  # Number of attributes

resnet18.fc = nn.Sequential(
    nn.Linear(in_features, num_attributes),
    nn.Sigmoid()
)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(resnet18.parameters(), lr=0.001)

In [ ]:
num_epochs = 10  # Adjust as needed

for epoch in range(num_epochs):
    resnet18.train()
    loss = 0
    for inputs, labels in tqdm(train_loader):
        optimizer.zero_grad()
        outputs = resnet18(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    # Print statistics or log them as needed
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss}')


 68%|███████████████████████████▋             | 106/157 [08:55<04:08,  4.87s/it]